<a href="https://colab.research.google.com/github/debernall/Proyecto/blob/main/Proyecto_bootcamp_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introducción

# Proyecto Lluvia

## Limpieza de archivos

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

*Sistema de archivos*

    ./
    
    ./Proyecto/
        ./Proyecto/Proyecto_bootcamp_cuaderno.ipynb

    ./Datasets/
        ./Datasets/...... .csv
    
    ./CleanDatasets/
        ./CleanDatasets/.....  .parquet


### Primera exploración

Se importa el primer documento previo a la limpieza. Se inicia con el archivo de menor tamaño.

In [2]:
df = pd.read_csv('../Datasets/Precipitaci_n_20241016(vaupes).csv',
                 sep=',',
                 #nrows=10
                 )
df

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,42077020,240,11/01/2023 07:30:00 PM,0.2,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
1,42077020,240,11/01/2023 11:10:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
2,42077020,240,11/01/2023 03:10:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
3,42077020,240,11/01/2023 09:30:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
4,42077020,240,11/01/2023 05:40:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
...,...,...,...,...,...,...,...,...,...,...,...,...
9147,42077020,240,06/22/2024 02:20:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
9148,42077020,240,06/22/2024 01:30:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
9149,42077020,240,06/22/2024 01:40:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm
9150,42077020,240,06/22/2024 01:20:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24,PRECIPITACIÓN,mm


En la siguiente celda se imprime un pequeño resumen. Es importante resaltar que no hay datos nulos y la necesidad de cambiar el formato de cada columna por uno mas adecuado que reduzca el tamaño del dataframe.

In [3]:
size_0=df.memory_usage(deep=True).sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9152 entries, 0 to 9151
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CodigoEstacion     9152 non-null   int64  
 1   CodigoSensor       9152 non-null   int64  
 2   FechaObservacion   9152 non-null   object 
 3   ValorObservado     9152 non-null   float64
 4   NombreEstacion     9152 non-null   object 
 5   Departamento       9152 non-null   object 
 6   Municipio          9152 non-null   object 
 7   ZonaHidrografica   9152 non-null   object 
 8   Latitud            9152 non-null   float64
 9   Longitud           9152 non-null   float64
 10  DescripcionSensor  9152 non-null   object 
 11  UnidadMedida       9152 non-null   object 
dtypes: float64(3), int64(2), object(7)
memory usage: 858.1+ KB


### Vaupes

Se importa nuevamente el archivo csv, seleccionando las columnas relevantes y eligiendo un formato adeacuado para cada columna.

In [4]:
df = pd.read_csv('../Datasets/Precipitaci_n_20241016(vaupes).csv',
                 sep=',',
                 usecols= [0,1,2,3,4,5,6,7,8,9],
                 dtype={0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category'}
                 )
df

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud
0,0042077020,0240,11/01/2023 07:30:00 PM,0.2,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
1,0042077020,0240,11/01/2023 11:10:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
2,0042077020,0240,11/01/2023 03:10:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
3,0042077020,0240,11/01/2023 09:30:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
4,0042077020,0240,11/01/2023 05:40:00 PM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
...,...,...,...,...,...,...,...,...,...,...
9147,0042077020,0240,06/22/2024 02:20:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
9148,0042077020,0240,06/22/2024 01:30:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
9149,0042077020,0240,06/22/2024 01:40:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24
9150,0042077020,0240,06/22/2024 01:20:00 AM,0.0,MITU,VAUPES,MITÚ,VAUPES,1.26,-70.24


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9152 entries, 0 to 9151
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   CodigoEstacion    9152 non-null   category
 1   CodigoSensor      9152 non-null   category
 2   FechaObservacion  9152 non-null   object  
 3   ValorObservado    9152 non-null   float32 
 4   NombreEstacion    9152 non-null   category
 5   Departamento      9152 non-null   category
 6   Municipio         9152 non-null   category
 7   ZonaHidrografica  9152 non-null   category
 8   Latitud           9152 non-null   category
 9   Longitud          9152 non-null   category
dtypes: category(8), float32(1), object(1)
memory usage: 178.9+ KB


Se muestra una comparación entre el tamaño del dataframe tras el cambio de formato a las columnas

In [6]:
size_f=df.memory_usage(deep=True).sum()
print(size_f/size_0)

0.1759772734217371


**Formato de fecha**

In [7]:
df['FechaObservacion'] = pd.to_datetime(df['FechaObservacion'], format='%m/%d/%Y %H:%M:%S %p')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9152 entries, 0 to 9151
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CodigoEstacion    9152 non-null   category      
 1   CodigoSensor      9152 non-null   category      
 2   FechaObservacion  9152 non-null   datetime64[ns]
 3   ValorObservado    9152 non-null   float32       
 4   NombreEstacion    9152 non-null   category      
 5   Departamento      9152 non-null   category      
 6   Municipio         9152 non-null   category      
 7   ZonaHidrografica  9152 non-null   category      
 8   Latitud           9152 non-null   category      
 9   Longitud          9152 non-null   category      
dtypes: category(8), datetime64[ns](1), float32(1)
memory usage: 178.9 KB


In [9]:
df.to_parquet('../CleanDatasets/rain_vaupes.parquet')

In [10]:
import re
import os
namesFiles = os.listdir('../Datasets')
namesFiles = [re.findall(r'\((\w*)\)',x)[0] for x in namesFiles]
print(namesFiles)

['amazonas', 'antioquia', 'arauca', 'atlantico1', 'atlantico2', 'bogota1', 'bogota2', 'bogota3', 'bolivar1', 'bolivar2', 'boyaca', 'caldas', 'caqueta1', 'caqueta2', 'casanare', 'cauca', 'cesar', 'choco', 'choco1', 'cordoba1', 'cordoba2', 'cundinamarca', 'guainia', 'guajira', 'guaviare', 'huila', 'magdalena', 'meta', 'narino', 'narino1', 'nill', 'norte_de_santander', 'putumayo', 'quindio', 'risaralda', 'sanandres1', 'sanandres2', 'sanandres3', 'sanandres4', 'santander', 'sucre', 'tolima', 'valle_del_cauca', 'vaupes', 'vaupes1', 'vichada']


### Caquetá

Tener en cuenta que la información sobre caquetá está almacenada en dos archivos

#### Caquetá 2

In [11]:
nameFile = '../Datasets/Precipitaci_n_20241016('+ namesFiles[13] +').csv'
nameFile

'../Datasets/Precipitaci_n_20241016(caqueta2).csv'

In [12]:
df_caqueta2 = pd.read_csv(nameFile,
                 sep=',',
                 usecols= [0,1,2,3,4,5,6,7,8,9],
                 dtype={0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category'}
                 )
df_caqueta2['FechaObservacion'] = pd.to_datetime(df_caqueta2['FechaObservacion'], format='%m/%d/%Y %H:%M:%S %p')
df_caqueta2

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud
0,0044055010,0240,2023-09-13 01:50:00,0.0,TRES ESQUINAS,CAQUETÁ,SOLANO,CAQUETÁ,0.7375,-75.23611111
1,0044035501,0240,2023-09-18 06:30:00,0.0,PAUJIL,CAQUETÁ,EL PAUJIL,CAQUETÁ,1.57381944,-75.3402
2,0044035501,0240,2023-09-19 07:50:00,0.0,PAUJIL,CAQUETÁ,EL PAUJIL,CAQUETÁ,1.57381944,-75.3402
3,0044035501,0240,2023-09-19 07:30:00,0.0,PAUJIL,CAQUETÁ,EL PAUJIL,CAQUETÁ,1.57381944,-75.3402
4,0044035501,0240,2023-09-18 09:20:00,0.0,PAUJIL,CAQUETÁ,EL PAUJIL,CAQUETÁ,1.57381944,-75.3402
...,...,...,...,...,...,...,...,...,...,...
14541,0044055010,0240,2024-06-12 02:30:00,0.0,TRES ESQUINAS,CAQUETÁ,SOLANO,CAQUETÁ,0.7375,-75.23611111
14542,0044055010,0240,2024-06-12 06:30:00,0.0,TRES ESQUINAS,CAQUETÁ,SOLANO,CAQUETÁ,0.7375,-75.23611111
14543,0044055010,0240,2024-06-12 12:50:00,0.0,TRES ESQUINAS,CAQUETÁ,SOLANO,CAQUETÁ,0.7375,-75.23611111
14544,0044055010,0240,2024-06-12 02:10:00,0.0,TRES ESQUINAS,CAQUETÁ,SOLANO,CAQUETÁ,0.7375,-75.23611111


La siguiente celda imprimirá la cantidad de valores de nulos

In [13]:
nulls = df_caqueta2.count(axis=1).sum()
size = df_caqueta2.shape[0]*df_caqueta2.shape[1]
print(size-nulls)

0


De nuevo se observa que el archivo tiene un tamaño menor y que los diferentes formatos son aplicados correctamente a cada columna

In [14]:
df_caqueta2.info()
#df_caqueta2.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14546 entries, 0 to 14545
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CodigoEstacion    14546 non-null  category      
 1   CodigoSensor      14546 non-null  category      
 2   FechaObservacion  14546 non-null  datetime64[ns]
 3   ValorObservado    14546 non-null  float32       
 4   NombreEstacion    14546 non-null  category      
 5   Departamento      14546 non-null  category      
 6   Municipio         14546 non-null  category      
 7   ZonaHidrografica  14546 non-null  category      
 8   Latitud           14546 non-null  category      
 9   Longitud          14546 non-null  category      
dtypes: category(8), datetime64[ns](1), float32(1)
memory usage: 285.1 KB


In [24]:
print(namesFiles)
print(len(namesFiles))

['amazonas', 'antioquia', 'arauca', 'atlantico1', 'atlantico2', 'bogota1', 'bogota2', 'bogota3', 'bolivar1', 'bolivar2', 'boyaca', 'caldas', 'caqueta1', 'caqueta2', 'casanare', 'cauca', 'cesar', 'choco', 'choco1', 'cordoba1', 'cordoba2', 'cundinamarca', 'guainia', 'guajira', 'guaviare', 'huila', 'magdalena', 'meta', 'narino', 'narino1', 'nill', 'norte_de_santander', 'putumayo', 'quindio', 'risaralda', 'sanandres1', 'sanandres2', 'sanandres3', 'sanandres4', 'santander', 'sucre', 'tolima', 'valle_del_cauca', 'vaupes', 'vaupes1', 'vichada']
46


In [25]:
for name in namesFiles[:]:
    nameFile = '../Datasets/Precipitaci_n_20241016('+ name +').csv'
    df_dept = pd.read_csv(nameFile,
                 sep=',',
                 usecols= [0,1,2,3,4,5,6,7,8,9],
                 dtype={0:'category',1:'category',2:'str',3:'float32',4:'category',5:'category',6:'category',7:'category',8:'category',9:'category'}
                 )
    df_dept['FechaObservacion'] = pd.to_datetime(df_dept['FechaObservacion'], format='%m/%d/%Y %H:%M:%S %p')
    nulls = df_dept.count(axis=1).sum()
    size = df_dept.shape[0]*df_dept.shape[1]
    if nulls-size != 0:
        print(f'{nulls-size} nulls in {name}, parquet file can not created')
    else:
        df_dept.to_parquet(f'../CleanDatasets/rain_{name}.parquet')


In [26]:
df_2 = pd.read_parquet(f'../CleanDatasets/rain_tolima.parquet')
df_2

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud
0,0021185501,0240,2018-05-11 05:45:00,0.0,ROVIRA - AUT,TOLIMA,ROVIRA,ALTO MAGDALENA,4.266666667,-75.28305556
1,0021137050,0240,2015-01-19 05:50:00,0.0,RIO ANGOSTURA - AUT,TOLIMA,NATAGAIMA,ALTO MAGDALENA,3.443194444,-75.11880556
2,0022057010,0240,2014-09-12 01:40:00,0.0,PIEDRAS DE COBRE - AUT,TOLIMA,SALDAÑA,SALDAÑA,3.908388889,-75.10327778
3,0021215180,0240,2012-02-28 01:40:00,0.0,BATALLON ROOKE - AUT,TOLIMA,IBAGUÉ,ALTO MAGDALENA,4.418388889,-75.24866667
4,2121500042,0240,2019-05-20 02:30:00,0.4,EL SILENCIO - AUT,TOLIMA,IBAGUÉ,ALTO MAGDALENA,4.600222222,-75.336
...,...,...,...,...,...,...,...,...,...,...
14540098,0021227010,0257,2024-10-15 07:20:00,0.0,PIEDRAS,TOLIMA,PIEDRAS,ALTO MAGDALENA,4.535225,-74.880530556
14540099,0022070030,0240,2024-10-15 02:20:00,0.0,SANTA HELENA,TOLIMA,RONCESVALLES,SALDAÑA,4.124555556,-75.49952778
14540100,0021185080,0240,2024-10-15 05:10:00,0.0,VALLE DE SAN JUAN,TOLIMA,VALLE DE SAN JUAN,ALTO MAGDALENA,4.216458333,-75.104611111
14540101,0021227010,0257,2024-10-15 06:10:00,0.0,PIEDRAS,TOLIMA,PIEDRAS,ALTO MAGDALENA,4.535225,-74.880530556
